In [1]:
from uat_suggestor import uat_manager
import pandas as pd
import numpy as np

df = pd.read_csv('uat_eval_dataset.csv')

df_subset = df.sample(n=100)
df_subset = df_subset.reset_index(drop=True)

Index uat-index-with-exp-decay has been initialized 
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\karthik rathod\.cache\huggingface\token
Login successful
embeddings/embeddings.npy already exists, no need to download.
embedding_data.csv already exists, no need to download.


c:\study_setup\Anaconda\envs\scrape_env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   bibcode       100 non-null    object
 1   abstract      100 non-null    object
 2   uat           100 non-null    object
 3   uat_keywords  100 non-null    object
 4   uat_lenght    100 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [6]:
import time

# List of columns we expect to have or create
expected_columns = [
    'type', 
    'output', 
    'list_of_branch', 
    'retreived_branches_abstract', 
    'retreived_branches_concepts', 
    'retreived_keywords_abstract'
]

# Create a copy of the original dataframe
df_subset_eval = df_subset.copy()

# Create missing columns
for column in expected_columns:
    if column not in df_subset_eval.columns:
        df_subset_eval[column] = None

# Process the dataframe
for i in range(len(df_subset_eval['abstract'])):
    max_attempts = 3
    attempt = 0
    success = False

    while attempt < max_attempts and not success:
        try:
            response = uat_manager(df_subset_eval['abstract'][i])            
            df_subset_eval.at[i, 'type'] = response['type']
            df_subset_eval.at[i, 'output'] = response['output']
            print(len(response['list_of_branch']))
            df_subset_eval.at[i, 'list_of_branch'] = response['list_of_branch']
            df_subset_eval.at[i, 'retreived_branches_abstract'] = response['retreived_branches_abstract']
            df_subset_eval.at[i, 'retreived_branches_concepts'] = response['retreived_branches_concepts']
            df_subset_eval.at[i, 'retreived_keywords_abstract'] = response['retreived_keywords_abstract']

            success = True
        except Exception as e:
            attempt += 1
            if attempt < max_attempts:
                print(f"Error occurred: {str(e)}. Retrying... (Attempt {attempt} of {max_attempts})")
                time.sleep(1)  # Wait for 1 second before retrying
            else:
                print(f"Error persisted after {max_attempts} attempts: {str(e)}")
                error_message = f"Error after {max_attempts} attempts: {str(e)}"
                for column in expected_columns:
                    df_subset_eval.at[i, column] = error_message

    # Optional: Print progress
    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1} out of {len(df_subset_eval)} rows")

10
10
10
10
10
10
10
10
Error occurred: Extra data: line 29 column 1 (char 1766). Retrying... (Attempt 1 of 3)
10
10
Processed 10 out of 100 rows
10
10
Error occurred: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 42). Retrying... (Attempt 1 of 3)
10
10
10
10
10
10
10
10
Processed 20 out of 100 rows
Error occurred: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 29). Retrying... (Attempt 1 of 3)
2
10
10
10
10
10
10
10
10
10
Processed 30 out of 100 rows
10
10
10
Error occurred: Extra data: line 2 column 1 (char 17). Retrying... (Attempt 1 of 3)
10
10
10
10
10
10
10
Processed 40 out of 100 rows
10
10
10
10
2
10
10
10
10
10
Processed 50 out of 100 rows
10
10
10
10
10
10
10
10
10
10
Processed 60 out of 100 rows
10
10
10
10
10
10
10
10
10
Error occurred: Expecting ',' delimiter: line 41 column 84 (char 2538). Retrying... (Attempt 1 of 3)
10
Processed 70 out of 100 rows
10
10
10
10
10
10
10
10
2
10
Processed 80 out of 100 rows
10
10
10
10
10
10
Error occurred: inv

In [7]:
df_subset_eval

,bibcode,abstract,uat,uat_keywords,uat_lenght,type,output,list_of_branch,retreived_branches_abstract,retreived_branches_concepts,retreived_keywords_abstract
0,2024ApJ...961L..13K,Potential contribution from gamma-ray sources ...,"['567', '628']","['Galactic cosmic rays', 'Gamma-ray astronomy']",2,reranked,UAT Branches in order of relevance:\n1. Branc...,[High energy astrophysics>Particle astrophysic...,[High energy astrophysics>Transient sources>Ga...,[Cosmology>Astronomical radiation sources>Gamm...,"{'concepts': [{'term': 'Gamma-ray Sources', 'e..."
1,2023ApJ...953....3H,We have measured line widths in active region ...,"['1989', '1485', '1988', '1992', '1995', '2038...","['Solar coronal heating', 'Solar coronal loops...",7,reranked,UAT Branches in order of relevance:\n1. Branc...,[Solar physics>Solar activity>Solar active reg...,[Solar physics>Solar activity>Solar active reg...,"[Solar physics>Helioseismology, Solar physics>...","{'concepts': [{'term': 'Coronal Loops', 'expla..."
2,2024AJ....167..137O,We report here the performance and first resul...,"['1277', '1278', '164', '870', '739', '1145']","['Polarimeters', 'Polarimetry', 'Blazars', 'Je...",6,reranked,UAT Branches in order of relevance:\n1. Branc...,[Observational astronomy>Astronomical instrume...,[Observational astronomy>Astronomical instrume...,[Observational astronomy>Astronomical instrume...,"{'concepts': [{'term': 'Optical Polarimetry', ..."
3,2022AJ....163..144G,"New sets of young M dwarfs with complex, sharp...","['1761', '1213', '1681', '1290', '982', '1629'...","['Variable stars', 'Periodic variable stars', ...",9,reranked,UAT Branches in order of relevance:\n1. Branc...,[Stellar astronomy>Stellar types>Variable star...,[Stellar astronomy>Stellar types>Variable star...,[Stellar astronomy>Stellar types>Stellar spect...,"{'concepts': [{'term': 'M dwarfs', 'explanatio..."
4,2021ApJ...916....6G,We investigate the origin of self-absorption i...,"['834', '352', '371']","['Interstellar clouds', 'Dark interstellar clo...",3,reranked,UAT Branches in order of relevance:\n1. Branc...,[Galactic and extragalactic astronomy>Galaxy p...,[Galactic and extragalactic astronomy>Galaxy p...,[Stellar astronomy>Stellar types>Stellar spect...,{'concepts': [{'term': '[O I] 63 μm line emiss...
...,...,...,...,...,...,...,...,...,...,...,...
95,2024RNAAS...8...86F,Photoevaporation is believed to dominate the r...,"['1711', '1916', '2284', '2301']","['Transits', 'Time series analysis', 'Ultravio...",4,reranked,UAT Branches in order of relevance:\n1. Branc...,[Exoplanet astronomy>Exoplanet structure>Exopl...,[Exoplanet astronomy>Exoplanet structure>Exopl...,[Observational astronomy>Astronomical methods>...,"{'concepts': [{'term': 'Photoevaporation', 'ex..."
96,2022ApJS..263...22H,Using ion measurements from Ultra-Low-Energy I...,"['18', '1491', '1988', '1477', '1474', '711', ...","['Active sun', 'Solar energetic particles', 'A...",7,reranked,UAT Branches in order of relevance:\n1. Branc...,[Solar physics>Solar particle emission>Solar e...,"[Solar physics>Solar abundances, Solar physics...",[Solar physics>Solar activity>Solar active reg...,{'concepts': [{'term': 'Solar Energetic Partic...
97,2020ApJ...893....1B,Our ability to study the properties of the int...,"['1031', '1736', '459', '574', '732']","['Metallicity', 'Ultraviolet astronomy', 'Emis...",5,reranked,UAT Branches in order of relevance:\n1. Branc...,[Galactic and extragalactic astronomy>Galaxy p...,[Galactic and extragalactic astronomy>Galaxy p...,[Interdisciplinary astronomy>Astrochemistry>Ch...,{'concepts': [{'term': 'Interstellar Medium (I...
98,2019ApJ...882....2V,We perform GR-MHD simulations of outflow launc...,"['14', '1964', '159', '17']","['Accretion', 'Magnetohydrodynamics', 'Black h...",4,reranked,UAT Branches in order of relevance:\n1. Branc...,[High energy astrophysics>Black hole physics>B...,[High energy astrophysics>Black hole physics>B...,[High energy astrophysics>Black hole physics>B...,"{'concepts': [{'term

In [8]:
df_subset_eval.to_csv('uat_eval_dataset_output_new.csv', index=False)

In [9]:
df_subset_eval.type.unique()

array(['reranked', 'new_concept', 'new_concept_exists'], dtype=object)

In [10]:
df_eval_reranked = df_subset_eval[df_subset_eval['type'] == 'reranked']
df_eval_reranked.to_csv('eval_reranked_new.csv', index=False)
df_eval_new_concept = df_subset_eval[df_subset_eval['type'] == 'new_concept']
df_eval_new_concept.to_csv('eval_new_concept_new.csv', index=False)
df_eval_new_concept_exist = df_subset_eval[df_subset_eval['type'] == 'new_concept_exist']
df_eval_new_concept_exist.to_csv('eval_new_concept_exist_new.csv', index=False)

In [12]:
df_eval_reranked = df_subset_eval[:10][df_subset_eval['type'] == 'reranked']
df_eval_reranked.to_csv('eval_reranked_mini.csv', index=False)

C:\Users\karthik rathod\AppData\Local\Temp\ipykernel_10992\3124411070.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_eval_reranked = df_subset_eval[:10][df_subset_eval['type'] == 'reranked']
